# Atacama soil microbiome” tutorial

**_note:_  This guide assumes you have QIIME 2 installed (e.g. using this [procedure](https://docs.qiime2.org/2019.10/install/native/)). To execute the script properly, open this notebook in a Jupyter Notebook from within a conda QIIME 2 environment.**

**_note:_  This tutorial is an adaptation of the same tutorial that may be found on the [official QIIME 2 docs website](ing-pictures/). The original tutorial uses the QIIME 2 CLI interface.**

Instead of CLI interface, this tutorial uses [Artifact API](https://docs.qiime2.org/2019.10/interfaces/artifact-api/) - a Python 3 application programmhttps://docs.qiime2.org/2019.10/tutorials/mover interface (API) for QIIME 2. The Artifact API supports interactive computing with QIIME 2 using the Python 3 programming language. The API is automatically generated, and its availability depends on which QIIME 2 plugins are currently installed. It has been optimized for use in the Jupyter Notebook. The Artifact API is a part of the QIIME 2 framework; no additional software needs to be installed to use it.

The notebook was tested using the ` 2020.2 ` version of QIIME 2.

Before you start: close this notebook and Jupyter session, and run `jupyter serverextension enable --py qiime2 --sys-prefix`. Then, restart this notebook.

This tutorial is designed to serve two purposes. First, it illustrates the initial processing steps of paired-end read analysis, up to the point where the analysis steps are identical to single-end read analysis. This includes the importing, demultiplexing, and denoising steps, and results in a feature table and the associated feature sequences. Second, this is intended to be a self-guided exercise that could be run after the [moving pictures tutorial](https://docs.qiime2.org/2020.2/tutorials/moving-pictures/) to gain more experience with QIIME 2. For this exercise, we provide some questions that can be used to guide your analysis, but do not provide commands that will allow you to address each. Instead, you should apply the commands that you learned in the [moving pictures tutorial](https://docs.qiime2.org/2020.2/tutorials/moving-pictures/).

In this tutorial you’ll use QIIME 2 to perform an analysis of soil samples from the Atacama Desert in northern Chile. The Atacama Desert is one of the most arid locations on Earth, with some areas receiving less than a millimeter of rain per decade. Despite this extreme aridity, there are microbes living in the soil. The soil microbiomes profiled in this study follow two east-west transects, Baquedano and Yungay, across which average soil relative humidity is positively correlated with elevation (higher elevations are less arid and thus have higher average soil relative humidity). Along these transects, pits were dug at each site and soil samples were collected from three depths in each pit.

# Importing necessary modules

In [3]:
import qiime2

In [4]:
from qiime2.plugins import demux, dada2, metadata, feature_table

In [5]:
from qiime2.plugins.demux.methods import emp_paired

# Creating a new directory

Create a directory to work in called `qiime2-atacama-tutorial` and change to that directory:

In [6]:
workdir='/home/user/Documents/qiime2-ATACAMA-tutorial/'

In [7]:
!mkdir -p $workdir
!cd $workdir

# Obtaining and importing data files

Before starting the analysis, explore the [sample metadata](https://docs.google.com/spreadsheets/d/1AFtHGlLIHy4-hwAyAL0EQUMLvZtONK5bgZ0JSInSRYc/edit#gid=0) to familiarize yourself with the samples used in this study. The sample metadata is available as a Google Sheet. This `sample-metadata.tsv` file is used throughout the rest of the tutorial.

In [6]:
!wget -O $workdir/"sample-metadata.tsv" \
                    "https://data.qiime2.org/2020.2/tutorials/atacama-soils/sample_metadata.tsv"

--2021-03-03 18:49:06--  https://data.qiime2.org/2020.2/tutorials/atacama-soils/sample_metadata.tsv
Resolving data.qiime2.org... 52.35.38.247
Connecting to data.qiime2.org|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/1AFtHGlLIHy4-hwAyAL0EQUMLvZtONK5bgZ0JSInSRYc/export?gid=0&format=tsv [following]
--2021-03-03 18:49:06--  https://docs.google.com/spreadsheets/d/1AFtHGlLIHy4-hwAyAL0EQUMLvZtONK5bgZ0JSInSRYc/export?gid=0&format=tsv
Resolving docs.google.com... 216.58.215.78, 2a00:1450:401b:802::200e
Connecting to docs.google.com|216.58.215.78|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-04-6o-sheets.googleusercontent.com/export/l5l039s6ni5uumqbsj9o11lmdc/bijtejtl2efmfir522bk555m2k/1614793745000/103995680502445084602/*/1AFtHGlLIHy4-hwAyAL0EQUMLvZtONK5bgZ0JSInSRYc?gid=0&format=tsv [following]
--2021-03-03 18:49:07--  https://doc-04-6o-sheets.googleus

Next, you’ll download the multiplexed reads. You will download three `fastq.gz` files, corresponding to the forward, reverse, and barcode (i.e., index) reads. These files contain a subset of the reads in the full data set generated for this study, which allows for the following commands to be run relatively quickly. 

Create a directory to work in called `emp-paired-end-sequences` and change to that directory:

In [8]:
!mkdir $workdir/emp-paired-end-sequences
!cd $workdir

mkdir: cannot create directory ‘/home/user/Documents/qiime2-ATACAMA-tutorial//emp-paired-end-sequences’: File exists


In [11]:
!wget -O $workdir/"emp-paired-end-sequences/forward.fastq.gz" \
  "https://data.qiime2.org/2020.11/tutorials/atacama-soils/10p/forward.fastq.gz"

--2021-03-03 18:58:06--  https://data.qiime2.org/2020.11/tutorials/atacama-soils/10p/forward.fastq.gz
Resolving data.qiime2.org... 52.35.38.247
Connecting to data.qiime2.org|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/tutorials/atacama-soils/10p/forward.fastq.gz [following]
--2021-03-03 18:58:07--  https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/tutorials/atacama-soils/10p/forward.fastq.gz
Resolving s3-us-west-2.amazonaws.com... 52.218.144.68
Connecting to s3-us-west-2.amazonaws.com|52.218.144.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143193967 (137M) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-ATACAMA-tutorial//emp-paired-end-sequences/forward.fastq.gz’

/home/user/Document 100%[===================>] 136.56M   237KB/s    in 9m 34s  

2021-03-03 19:07:43 (243 KB/s) - ‘/home/user/Documents/qiime2-ATACAMA-tutorial//emp-paired-e

In [12]:
!wget -O $workdir/"emp-paired-end-sequences/reverse.fastq.gz" \
  "https://data.qiime2.org/2020.11/tutorials/atacama-soils/10p/reverse.fastq.gz"

--2021-03-03 19:07:53--  https://data.qiime2.org/2020.11/tutorials/atacama-soils/10p/reverse.fastq.gz
Resolving data.qiime2.org... 52.35.38.247
Connecting to data.qiime2.org|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/tutorials/atacama-soils/10p/reverse.fastq.gz [following]
--2021-03-03 19:07:54--  https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/tutorials/atacama-soils/10p/reverse.fastq.gz
Resolving s3-us-west-2.amazonaws.com... 52.218.136.232
Connecting to s3-us-west-2.amazonaws.com|52.218.136.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161032441 (154M) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-ATACAMA-tutorial//emp-paired-end-sequences/reverse.fastq.gz’

/home/user/Document 100%[===================>] 153.57M   829KB/s    in 5m 20s  

2021-03-03 19:13:15 (492 KB/s) - ‘/home/user/Documents/qiime2-ATACAMA-tutorial//emp-paired

In [13]:
!wget -O $workdir/"emp-paired-end-sequences/barcodes.fastq.gz" \
  "https://data.qiime2.org/2020.11/tutorials/atacama-soils/10p/barcodes.fastq.gz"

--2021-03-03 19:13:15--  https://data.qiime2.org/2020.11/tutorials/atacama-soils/10p/barcodes.fastq.gz
Resolving data.qiime2.org... 52.35.38.247
Connecting to data.qiime2.org|52.35.38.247|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/tutorials/atacama-soils/10p/barcodes.fastq.gz [following]
--2021-03-03 19:13:16--  https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/tutorials/atacama-soils/10p/barcodes.fastq.gz
Resolving s3-us-west-2.amazonaws.com... 52.218.185.120
Connecting to s3-us-west-2.amazonaws.com|52.218.185.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19976093 (19M) [binary/octet-stream]
Saving to: ‘/home/user/Documents/qiime2-ATACAMA-tutorial//emp-paired-end-sequences/barcodes.fastq.gz’

/home/user/Document 100%[===================>]  19.05M   850KB/s    in 24s     

2021-03-03 19:13:41 (827 KB/s) - ‘/home/user/Documents/qiime2-ATACAMA-tutorial//emp-pair

# Importing data as a qiime2 artifact

All data that is used as input to QIIME 2 is in form of QIIME 2 artifacts, which contain information about the type of data and the source of the data. So, the first thing we need to do is import these sequence data files into a QIIME 2 artifact.

In [9]:
sample_metadata = qiime2.Metadata.load(workdir+'/sample-metadata.tsv')

# Paired-end read analysis commands

To analyze these data, the sequences that you just downloaded must first be imported into an artifact of type `EMPPairedEndSequences`

In [10]:
paired_end_sequences = qiime2.Artifact.import_data('EMPPairedEndSequences', workdir+'/emp-paired-end-sequences/')

You next can demultiplex the sequence reads. This requires the sample metadata file, and you must indicate which column in that file contains the per-sample barcodes. In this case, that column name is `barcode-sequence`. In this data set, the barcode reads are the reverse complement of those included in the sample metadata file, so we additionally include the `rev_comp_mapping_barcodes` parameter.

After demultiplexing, we can generate and view a summary of how many sequences were obtained per sample.

In [11]:
demux_sequences = demux.methods.emp_paired(paired_end_sequences,
                                           sample_metadata.get_column('barcode-sequence'),
                                           rev_comp_mapping_barcodes = True)

Let’s subsample the data. We will perform this subsampling in this tutorial for two reasons - one, to speed up the tutorial run time, and two, to demonstrate the functionality.

In [12]:
demux_subsample = demux.methods.subsample_paired(demux_sequences.per_sample_sequences,
                                                 fraction = 0.3)

Let’s take a look at the summary in `demux-subsample.qzv`. In the “Per-sample sequence counts” table on the “Overview” tab, there are 75 samples in the data. If we look at the last 20 or so rows in the table, though, we will observe that many samples have fewer than 100 reads in them - let’s filter those samples out of the data:

In [13]:
demux_summary = demux.visualizers.summarize(demux_subsample.subsampled_sequences)
demux_summary.visualization

/home/user/miniconda2/envs/qiime2-2021.2/lib/python3.6/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/home/user/miniconda2/envs/qiime2-2021.2/lib/python3.6/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<visualization: Visualization uuid: 0a60dd45-338b-4432-9279-94a012eb27db>

After demultiplexing reads, we’ll look at the sequence quality based on ten-thousand randomly selected reads, and then denoise the data. When you view the quality plots, note that in contrast to the corresponding plots in the [moving pictures tutorial](https://docs.qiime2.org/2020.2/tutorials/moving-pictures/), there are now two interactive plots to be considered together. The plot on the left presents the quality scores for the forward reads, and the plot on the right presents the quality scores for the reverse reads. We’ll use these plots to determine what trimming parameters we want to use for denoising with [DADA2](https://www.ncbi.nlm.nih.gov/pubmed/27214047), and then denoise the reads using dada2 denoise-paired.

In this example we have 150-base forward and reverse reads. Since we need the reads to be long enough to overlap when joining paired ends, the first thirteen bases of the forward and reverse reads are being trimmed, but no trimming is being applied to the ends of the sequences to avoid reducing the read length by too much. In this example, the same values are being provided for `trim_left_f` and `trim_left_r` and for `trunc_len_f` and `p_trunc_len_r`, but that is not a requirement.

In [14]:
denoised_sequences = dada2.methods.denoise_paired(demux_sequences.per_sample_sequences,
                                                  trim_left_f = 13,
                                                  trim_left_r = 13,
                                                  trunc_len_f = 150,
                                                  trunc_len_r = 150)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /tmp/tmp5clwo9jq/forward /tmp/tmp5clwo9jq/reverse /tmp/tmp5clwo9jq/output.tsv.biom /tmp/tmp5clwo9jq/track.tsv /tmp/tmp5clwo9jq/filt_f /tmp/tmp5clwo9jq/filt_r 150 150 13 13 2.0 2.0 2 independent consensus 1.0 1 1000000



At this stage, you will have artifacts containing the feature table and corresponding feature sequences. You can generate summaries of those as follows.

In [15]:
output_viz = feature_table.visualizers.summarize(denoised_sequences.table)
output_viz.visualization

/home/user/miniconda2/envs/qiime2-2021.2/lib/python3.6/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/home/user/miniconda2/envs/qiime2-2021.2/lib/python3.6/site-packages/seaborn/distributions.py:2056: FutureWarning: The `axis` variable is no longer used and will be removed. Instead, assign variables directly to `x` or `y`.
  warnings.warn(msg, FutureWarning)
/home/user/miniconda2/envs/qiime2-2021.2/lib/python3.6/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.war

<visualization: Visualization uuid: 635b2cfe-9632-4c5f-8b0e-b44201694d91>

In [16]:
rep_seq = metadata.visualizers.tabulate(denoised_sequences.representative_sequences.view(qiime2.Metadata))
rep_seq.visualization

<visualization: Visualization uuid: 9355ec25-ee5c-48fe-b33a-aaa017689fc4>

As well, you can visualize the denoising stats by running:

In [17]:
denoised_stats = metadata.visualizers.tabulate(denoised_sequences.denoising_stats.view(qiime2.Metadata))
denoised_stats.visualization

<visualization: Visualization uuid: 8400e959-86d6-409c-a05d-74c09f4c2756>

From this point, analysis of paired-end read data progresses in the same way as analysis of single-end read data. You can therefore continue your analyses of these data following the steps that you ran in the moving pictures tutorial.

# Questions to guide data analysis¶


Use the following questions to guide your further analyses of these data data.

1. What value would you choose to pass for `sampling_depth` ? How many samples will be excluded from your analysis based on this choice? Approximately how many total sequences will you be analyzing in the `core_metrics_phylogenetic` command?
2. What sample metadata or combinations of sample metadata are most strongly associated with the differences in robial composition of the samples? Are these associations stronger with unweighted UniFrac or with Bray-Curtis? Based on what you know about these metrics, what does that difference suggest? For exploring associations between continuous metadata and sample composition, the commands `import distance_matrix` in combination with `mantel` and `bioenv` will be useful. These were not covered in the Moving Pictures tutorial, but you can learn about them by running them by pressing the `Shift + TAB`.
3. What do you conclude about the associations between continuous sample metadata and the richness and evenness of these samples? For exploring associations between continuous metadata and richness or evenness, the command `alpha_correlation` will be useful. This was not covered in the Moving Pictures tutorial, but you can learn about it by pressing the `Shift + TAB`.
4. Which categorical sample metadata columns are most strongly associated with the differences in microbial     community richness or evenness? Are these differences statistically significant?\n",
5. In taxonomic composition bar plots, sort the samples by their average soil relative humidity, and visualize them at the phylum level. What are the dominant phyla in these samples? Which phyla increase and which decrease with increasing average soil relative humidity?\n",
6. What phyla differ in abundance across vegetated and unvegetated sites?

The data used in this tutorial is presented in: `Significant Impacts of Increasing Aridity on the Arid Soil Microbiome. Julia W. Neilson, Katy Califf, Cesar Cardona, Audrey Copeland, Will van Treuren, Karen L. Josephson, Rob Knight, Jack A. Gilbert, Jay Quade, J. Gregory Caporaso, and Raina M. Maier. mSystems May 2017, 2 (3) e00195-16; DOI: 10.1128/mSystems.00195-16`.